In [182]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import numpy as np

In [183]:
num=2
Ori = "Lactobacillus_rhamnosus_NZ_CP006804.1_2"
target = " ".join(Ori.split("_")[0:2])
Ori_seq = "_".join(Ori.split("_")[2:5])
url = "https://www.ncbi.nlm.nih.gov/tools/primer-blast/primertool.cgi?ctg_time=1653568013&job_key=KiD128AtzYXqu8i-xd7sjL_F_b6S1uajkw"
r =requests.get(url)
soup = BeautifulSoup(r.content,"lxml")

In [184]:
data_list = []
for i in soup.find('div',{'id':'alignments'}).children:
    for n in i.find_all_next(attrs={"class":"prPairInfo"}):
        data_list.append(n.get_text())

In [185]:
result_list = []
for data in data_list:
    file = [x for x in data.split("\n") if x]
    Primer = file[0]
    FPrimer = [re.findall("(?<=Forward primer)(\w+)Plus",x)[0]for x in file if re.findall("(?<=Forward primer)(\w+)Plus",x)][0]
    RPrimer = [re.findall("(?<=Reverse primer)(\w+)Minus",x)[0]for x in file if re.findall("(?<=Reverse primer)(\w+)Minus",x)][0]
    all_species = [" ".join(x.split(",")[0].split(" ")[2:4]) for x in file if x.startswith(">")]
    target_species = [x for x in all_species if re.findall(target,x)]
    length = [int(re.findall("(?<=product length = )(\d+)",x)[0]) for x in file if re.findall("(?<=product length = )(\d+)",x)]
    mean_len = round(np.mean(length),3)
    std_length = round(np.std(length),3)
    result_list.append([Primer,Ori_seq,target,FPrimer,RPrimer,len(all_species),len(target_species),mean_len,std_length,len(set(all_species)),",".join(set(all_species))])

In [186]:
df_out = pd.DataFrame(result_list,columns=["Primer_Name","Ori","Target","FP","RP","SP_Num","Target_Num","Mean_Len","Std_Len","All_SP_Num","ALL_SP"]).drop_duplicates()

In [187]:
df_out

,Primer_Name,Ori,Target,FP,RP,SP_Num,Target_Num,Mean_Len,Std_Len,All_SP_Num,ALL_SP
0,Primer pair 1,NZ_CP006804.1_2,Lactobacillus rhamnosus,CGGAGCATAAGTGGCCTTGA,CTGTTCACGCTCTGCTCTGA,3,1,1054.35,1288.708,2,"Lactobacillus rhamnosus,Lacticaseibacillus rha..."
1,Primer pair 2,NZ_CP006804.1_2,Lactobacillus rhamnosus,TAAGTGGCCTTGAGCGTGAT,TGTTCACGCTCTGCTCTGAA,2,1,1315.50,1351.217,2,"Lactobacillus rhamnosus,Lacticaseibacillus rha..."
2,Primer pair 3,NZ_CP006804.1_2,Lactobacillus rhamnosus,TGCGCTCAAGGTCCTTATGT,GCTGAGGCCACTTACACTCC,1,0,1440.20,1390.509,1,Lacticaseibacillus rhamnosus
3,Primer pair 4,NZ_CP006804.1_2,Lactobacillus rhamnosus,AAGTGGCCTTGAGCGTGATG,TCTAAGCGCTCCTGTTCACG,1,0,340.85,626.469,1,Lacticaseibacillus rhamnosus
4,Primer pair 5,NZ_CP006804.1_2,Lactobacillus rhamnosus,ATAAGTGGCCTTGAGCGTGA,TCACGCTCTGCTCTGAAACG,1,0,1065.30,1261.137,1,Lacticaseibacillus rhamnosus
5,Primer pair 6,NZ_CP006804.1_2,Lactobacillus rhamnosus,TTGCGCTCAAGGTCCTTATGT,GGCTGAGGCCACTTACACTC,1,0,1486.35,1430.488,1,Lacticaseibacillus rhamnosus
6,Primer pair 7,NZ_CP006804.1_2,Lactobacillus rhamnosus,TGTGCAGGTTTCTGGACTGG,CCTCAGTCGCAATGGCCAAA,1,0,716.60,1068.307,1,Lacticaseibacillus rhamnosus
7,Primer pair 8,NZ_CP006804.1_2,Lactobacillus rhamnosus,CATAAGTGGCCTTGAGCGTG,CCTGTTCACGCTCTGCTCTG,3,1,1765.05,1486.826,2,"Lactobacillus rhamnosus,Lacticaseibacillus rha..."
8,Primer pair 9,NZ_CP006804.1_2,Lactobacillus rhamnosus,GCATAAGTGGCCTTGAGCGT,CTCCTGTTCACGCTCTGCTC,3,1,1020.50,1302.552,2,"Lactobacillus rhamnosus,Lacticaseibacillus rha..."
9,Primer pair 10,NZ_CP006804.1_2,Lactobacillus rhamnosus,AGTGGCCTTGAGCGTGATG,TGAGGCCACTTACACTCCGA,1,0,840.45,1033.916,1,Lacticaseibacillus rhamnosus


In [181]:
df_out.to_csv("/Volumes/Flower0501/Primer_Prediction/data/Lac/Primer/%s_NR_Primer_%d.txt"%(target,num),sep="\t",index=False)